In [1]:
!pip install catboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 8.6 MB/s eta 0:00:00


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from catboost import CatBoostClassifier, Pool
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import KFold,cross_val_score
from sklearn.model_selection import GridSearchCV

In [3]:
# Importación de archivos publicos en mega

!apt-get install megatools

!megadl 'https://mega.nz/file/IYR13BIL#xJGXa9RVya2lCZa97LnMcuyc-DJ8eSg_QrL9LwAPeGc'
!megadl 'https://mega.nz/file/sBYlFT5S#dyje3tq74m--QAOs9B60zR-f_9yBnUT54lYfjNZ2Rus'

# Lectura archivos
df = pd.read_csv('train.csv')
df2 = pd.read_csv('test.csv')

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  megatools
0 upgraded, 1 newly installed, 0 to remove and 49 not upgraded.
Need to get 207 kB of archives.
After this operation, 898 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 megatools amd64 1.10.3-1build1 [207 kB]
Fetched 207 kB in 0s (1,509 kB/s)
Selecting previously unselected package megatools.
(Reading database ... 123614 files and directories currently installed.)
Preparing to unpack .../megatools_1.10.3-1build1_amd64.deb ...
Unpacking megatools (1.10.3-1build1) ...
Setting up megatools (1.10.3-1build1) ...
Processing triggers for man-db (2.10.2-1) ...
Downloaded train.csv
Downloaded test.csv


#Preprocesado - Rellenar datos nulos

In [4]:
df.head(5)

,ID,PERIODO,ESTU_PRGM_ACADEMICO,ESTU_PRGM_DEPARTAMENTO,ESTU_VALORMATRICULAUNIVERSIDAD,ESTU_HORASSEMANATRABAJA,FAMI_ESTRATOVIVIENDA,FAMI_TIENEINTERNET,FAMI_EDUCACIONPADRE,FAMI_EDUCACIONMADRE,ESTU_PAGOMATRICULAPROPIO,RENDIMIENTO_GLOBAL
0,904256,20212,ENFERMERIA,BOGOTÁ,Entre 5.5 millones y menos de 7 millones,Menos de 10 horas,Estrato 3,Si,Técnica o tecnológica incompleta,Postgrado,No,medio-alto
1,645256,20212,DERECHO,ATLANTICO,Entre 2.5 millones y menos de 4 millones,0,Estrato 3,No,Técnica o tecnológica completa,Técnica o tecnológica incompleta,No,bajo
2,308367,20203,MERCADEO Y PUBLICIDAD,BOGOTÁ,Entre 2.5 millones y menos de 4 millones,Más de 30 horas,Estrato 3,Si,Secundaria (Bachillerato) completa,Secundaria (Bachillerato) completa,No,bajo
3,470353,20195,ADMINISTRACION DE EMPRESAS,SANTANDER,Entre 4 millones y menos de 5.5 millones,0,Estrato 4,Si,No sabe,Secundaria (Bachillerato) completa,No,alto
4,989032,20212,PSICOLOGIA,ANTIOQUIA,Entre 2.5 millones y menos de 4 millones,Entre 21 y 30 horas,Estrato 3,Si,Primaria completa,Primaria completa,No,medio-bajo


In [5]:
print(df.isnull().sum())

ID                                    0
PERIODO                               0
ESTU_PRGM_ACADEMICO                   0
ESTU_PRGM_DEPARTAMENTO                0
ESTU_VALORMATRICULAUNIVERSIDAD     6287
ESTU_HORASSEMANATRABAJA           30857
FAMI_ESTRATOVIVIENDA              32137
FAMI_TIENEINTERNET                26629
FAMI_EDUCACIONPADRE               23178
FAMI_EDUCACIONMADRE               23664
ESTU_PAGOMATRICULAPROPIO           6498
RENDIMIENTO_GLOBAL                    0
dtype: int64


In [6]:
print(df['ESTU_VALORMATRICULAUNIVERSIDAD'].value_counts())

ESTU_VALORMATRICULAUNIVERSIDAD
Entre 1 millón y menos de 2.5 millones      204048
Entre 2.5 millones y menos de 4 millones    127430
Menos de 500 mil                             80263
Entre 500 mil y menos de 1 millón            78704
Entre 4 millones y menos de 5.5 millones     69736
Más de 7 millones                            68014
Entre 5.5 millones y menos de 7 millones     38490
No pagó matrícula                            19528
Name: count, dtype: int64


In [7]:
# Cambio valores nulos en matricula a No pago matricula
df['ESTU_VALORMATRICULAUNIVERSIDAD'] = df['ESTU_VALORMATRICULAUNIVERSIDAD'].fillna("No pagó matrícula")
df2['ESTU_VALORMATRICULAUNIVERSIDAD'] = df2['ESTU_VALORMATRICULAUNIVERSIDAD'].fillna("No pagó matrícula")

In [8]:
print(df['ESTU_HORASSEMANATRABAJA'].value_counts())

ESTU_HORASSEMANATRABAJA
Más de 30 horas        249352
0                      116550
Entre 11 y 20 horas    115857
Entre 21 y 30 horas     92693
Menos de 10 horas       87191
Name: count, dtype: int64


In [9]:
# Cambio valores nulos a el más común Más de 30 horas
df['ESTU_HORASSEMANATRABAJA'] = df['ESTU_HORASSEMANATRABAJA'].fillna('Más de 30 horas')
df2['ESTU_HORASSEMANATRABAJA'] = df2['ESTU_HORASSEMANATRABAJA'].fillna('Más de 30 horas')

In [10]:
print(df['FAMI_ESTRATOVIVIENDA'].value_counts())

FAMI_ESTRATOVIVIENDA
Estrato 2      232671
Estrato 3      210685
Estrato 1      111991
Estrato 4       65514
Estrato 5       23608
Estrato 6       12605
Sin Estrato      3289
Name: count, dtype: int64


In [11]:
# Cambio valores nulos a el más común Estrato 2
df['FAMI_ESTRATOVIVIENDA'] = df['FAMI_ESTRATOVIVIENDA'].fillna('Estrato 2')
df2['FAMI_ESTRATOVIVIENDA'] = df2['FAMI_ESTRATOVIVIENDA'].fillna('Estrato 2')

In [12]:
print(df['FAMI_TIENEINTERNET'].value_counts())

FAMI_TIENEINTERNET
Si    592514
No     73357
Name: count, dtype: int64


In [13]:
# Cambio valores nulos a el más común Si
df['FAMI_TIENEINTERNET'] = df['FAMI_TIENEINTERNET'].fillna('Si')
df2['FAMI_TIENEINTERNET'] = df2['FAMI_TIENEINTERNET'].fillna('Si')

In [14]:
print(df['FAMI_EDUCACIONPADRE'].value_counts())

FAMI_EDUCACIONPADRE
Secundaria (Bachillerato) completa      128289
Primaria incompleta                     125675
Educación profesional completa           83117
Secundaria (Bachillerato) incompleta     71654
Técnica o tecnológica completa           62995
Primaria completa                        55958
Postgrado                                44169
Educación profesional incompleta         27084
Técnica o tecnológica incompleta         22552
Ninguno                                  22008
No sabe                                  16592
No Aplica                                 9229
Name: count, dtype: int64


In [15]:
# Cambio valores nulos a no aplica
df['FAMI_EDUCACIONPADRE'] = df['FAMI_EDUCACIONPADRE'].fillna('No Aplica')
df2['FAMI_EDUCACIONPADRE'] = df2['FAMI_EDUCACIONPADRE'].fillna('No Aplica')

In [16]:
print(df['FAMI_EDUCACIONMADRE'].value_counts())

FAMI_EDUCACIONMADRE
Secundaria (Bachillerato) completa      141744
Primaria incompleta                      99420
Técnica o tecnológica completa           89542
Educación profesional completa           85326
Secundaria (Bachillerato) incompleta     81012
Primaria completa                        56125
Postgrado                                46246
Técnica o tecnológica incompleta         27533
Educación profesional incompleta         22470
Ninguno                                  14483
No sabe                                   3017
No Aplica                                 1918
Name: count, dtype: int64


In [17]:
# Cambio valores nulos a no aplica
df['FAMI_EDUCACIONMADRE'] = df['FAMI_EDUCACIONMADRE'].fillna('No Aplica')
df2['FAMI_EDUCACIONMADRE'] = df2['FAMI_EDUCACIONMADRE'].fillna('No Aplica')

In [18]:
print(df['ESTU_PAGOMATRICULAPROPIO'].value_counts())

ESTU_PAGOMATRICULAPROPIO
No    382201
Si    303801
Name: count, dtype: int64


In [19]:
# Cambio valores nulos a el más común No
df['ESTU_PAGOMATRICULAPROPIO'] = df['ESTU_PAGOMATRICULAPROPIO'].fillna('No')
df2['ESTU_PAGOMATRICULAPROPIO'] = df2['ESTU_PAGOMATRICULAPROPIO'].fillna('No')

In [20]:
print(df.isnull().sum())

ID                                0
PERIODO                           0
ESTU_PRGM_ACADEMICO               0
ESTU_PRGM_DEPARTAMENTO            0
ESTU_VALORMATRICULAUNIVERSIDAD    0
ESTU_HORASSEMANATRABAJA           0
FAMI_ESTRATOVIVIENDA              0
FAMI_TIENEINTERNET                0
FAMI_EDUCACIONPADRE               0
FAMI_EDUCACIONMADRE               0
ESTU_PAGOMATRICULAPROPIO          0
RENDIMIENTO_GLOBAL                0
dtype: int64


In [21]:
df2.reset_index(drop=True, inplace=True)
df2.drop(columns=['Unnamed: 0'], inplace=True)

In [22]:
print(df2.isnull().sum())

ID                                0
PERIODO                           0
ESTU_PRGM_ACADEMICO               0
ESTU_PRGM_DEPARTAMENTO            0
ESTU_VALORMATRICULAUNIVERSIDAD    0
ESTU_HORASSEMANATRABAJA           0
FAMI_ESTRATOVIVIENDA              0
FAMI_TIENEINTERNET                0
FAMI_EDUCACIONPADRE               0
FAMI_EDUCACIONMADRE               0
ESTU_PAGOMATRICULAPROPIO          0
dtype: int64


In [23]:
df2.head(15)

,ID,PERIODO,ESTU_PRGM_ACADEMICO,ESTU_PRGM_DEPARTAMENTO,ESTU_VALORMATRICULAUNIVERSIDAD,ESTU_HORASSEMANATRABAJA,FAMI_ESTRATOVIVIENDA,FAMI_TIENEINTERNET,FAMI_EDUCACIONPADRE,FAMI_EDUCACIONMADRE,ESTU_PAGOMATRICULAPROPIO
0,550236,20183,TRABAJO SOCIAL,BOLIVAR,Menos de 500 mil,Menos de 10 horas,Estrato 3,Si,Técnica o tecnológica completa,Primaria completa,Si
1,98545,20203,ADMINISTRACION COMERCIAL Y DE MERCADEO,ANTIOQUIA,Entre 2.5 millones y menos de 4 millones,Entre 21 y 30 horas,Estrato 2,Si,Secundaria (Bachillerato) completa,Técnica o tecnológica completa,No
2,499179,20212,INGENIERIA MECATRONICA,BOGOTÁ,Entre 1 millón y menos de 2.5 millones,0,Estrato 3,Si,Secundaria (Bachillerato) incompleta,Secundaria (Bachillerato) completa,No
3,782980,20195,CONTADURIA PUBLICA,SUCRE,Entre 1 millón y menos de 2.5 millones,Entre 21 y 30 horas,Estrato 1,No,Primaria incompleta,Primaria incompleta,No
4,785185,20212,ADMINISTRACION DE EMPRESAS,ATLANTICO,Entre 2.5 millones y menos de 4 millones,Entre 11 y 20 horas,Estrato 2,Si,Secundaria (Bachillerato) completa,Secundaria (Bachillerato) completa,No
5,58495,20183,ADMINISTRACION AGROPECUARIA,VALLE,Entre 500 mil y menos de 1 millón,Más de 30 horas,Estrato 1,Si,Ninguno,Secundaria (Bachillerato) completa,Si
6,705444,20195,INGENIERIA CIVIL,SANTANDER,Menos de 500 mil,0,Estrato 3,Si,Primaria completa,Técnica o tecnológica completa,No
7,557548,20183,INGENIERÍA MECÁNICA,BOGOTÁ,Más de 7 millones,Menos de 10 horas,Estrato 4,Si,Postgrado,Postgrado,No
8,519909,20195,INGENIERÍA DE SISTEMAS,ATLANTICO,Entre 1 millón y menos de 2.5 millones,Entre 21 y 30 horas,Estrato 2,No,Secundaria (Bachillerato) completa,Secundaria (Bachillerato) completa,No
9,832058,20183,ECONOMIA,BOGOTÁ,Entre 500 mil y menos de 1 millón,Entre 21 y 30 horas,Estrato 2,Si,Técnica o tecnológica completa,Secundaria (Bachillerato) incompleta,No


#Entrenamiento del Modelo CatBoost


In [24]:
#Division de los datos
X_train = df.drop('RENDIMIENTO_GLOBAL', axis=1)
y_train = df['RENDIMIENTO_GLOBAL']

In [ ]:
#Tarda más de 1 hora 
param_grid = {
    'iterations': [500, 800],
    'learning_rate': [0.01, 0.1],
    'depth': [4, 6, 8],
    'l2_leaf_reg': [1, 3, 5]
}

grid_search = GridSearchCV(estimator=model, param_grid=param_grid, scoring='accuracy', cv=5, n_jobs=-1)
grid_search.fit(X_train, y_train)

In [ ]:
cv_results = grid_search.cv_results_
for mean_score, std, params in zip(cv_results["mean_test_score"], cv_results["std_test_score"], cv_results["params"]):
    print(f"Precisión: {mean_score:.4f}, Desviación: {std:.4f} para los parámetros {params}" )

In [25]:
#Entrenamiento del mejor modelo a ojo del desarrollador:
#(Mejor accuracy y varianza, a su vez que menos complejidad para que así sea más rapido el modelo)

categorical_features = ['PERIODO', 'ESTU_PRGM_ACADEMICO', 'ESTU_PRGM_DEPARTAMENTO',
                        'ESTU_VALORMATRICULAUNIVERSIDAD', 'ESTU_HORASSEMANATRABAJA',
                        'FAMI_ESTRATOVIVIENDA', 'FAMI_TIENEINTERNET',
                        'FAMI_EDUCACIONPADRE', 'FAMI_EDUCACIONMADRE',
                        'ESTU_PAGOMATRICULAPROPIO']

train_pool = Pool(X_train, y_train, cat_features=categorical_features)

model = CatBoostClassifier(iterations=800,
                           learning_rate=0.1,
                           depth=5,
                           cat_features=categorical_features,
                           verbose=100)

In [ ]:
# Hacer predicciones
predictions = model.predict(X_train)

# Evaluar el modelo con datos train
accuracy = accuracy_score(y_train, predictions)
print(f'Accuracy: {accuracy:.2f}')
print(classification_report(y_train, predictions))

##Aplicar el modelo a los datos test

In [ ]:
y_pred = model.predict(df2)

# Agregar la variable que se predijo a los datos
df2['RENDIMIENTO_GLOBAL'] = y_pred.ravel()

In [ ]:
#Preparar envio
predictions_df = df2[['ID', 'RENDIMIENTO_GLOBAL']]

# Guardar DataFrame
predictions_df.to_csv('submission.csv', index=False)